In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import molmass

In [ ]:
@np.vectorize
def atomic_number(ele):
    if ele == "Neutron":
        return 0
    return molmass.ELEMENTS[ele].number

In [ ]:
@np.vectorize
def get_A(iso):
    ele, A = iso.split("-")
    A = A.split("m")[0]
    
    return np.int(A)

In [ ]:
@np.vectorize
def get_ele(iso):
    ele, A = iso.split("-")
    return ele

## P16

Pignatari et al. (2016) reports yields in terms of over production factors (OP), defined as
$$
{\rm OP} = \frac{M_{\rm ej, i}}{M_{\rm ej} Z_{i}^0}
$$
 i.e. the ratio between the mean ejected abundance composition and the initial composition of the star. 
 Since we are interested in net fractional yields, we can calculate these from the overproduction factor as follows
 $$
 Y_{X} = Z_{\rm ini} ({\rm OP} - 1) \frac{M_{\rm ej}}{M_{\rm ini}} = (Z_{\rm ej} - Z_{\rm ini}) \frac{M_{\rm ej}}{M_{\rm ini}}
 $$.

Or 
$$
OP = 1 + \frac{Y_X}{Z_{\rm ini}} \frac{M_{\rm ini}}{M_{\rm ej}}
$$
 
Thus from each nu-grid model, we need the OP factor, the initial and ejected mass, and the birth composition.

In [ ]:
molmass = pd.read_csv(datadir + "nugrid/mesa_atomic_weights.txt", sep="\t", comment = "#")
molmass.element = molmass.element.str.title()
molmass["atomic_number"] = np.int64(np.arange(1, len(molmass)+1))

molmass.set_index("element", inplace=True)

In [ ]:
solar_Z = pd.read_csv(datadir + "nugrid/grevesse_noels_1993.txt", comment="!", sep="\t")
solar_Z.element = solar_Z.element.str.strip()

solar_Z.set_index("element", inplace=True)

In [ ]:
solar_Z["atomic_weight"] = [molmass.atomic_weight[ele] for ele, r in solar_Z.iterrows()]

In [ ]:
Y_H

In [ ]:
Ysun = 0.2485
Zsun = 0.0179

Y_H = 1 - (Zsun + Ysun) / molmass.atomic_weight["H"]
print(Y_H)
Y_i = Y_H * 10**(solar_Z.A - 12)
solar_Z["Z"] = Y_i * solar_Z.atomic_weight

print(sum(solar_Z.Z[2:]) - Zsun)
solar_Z

In [ ]:
P16_stars = pd.read_csv(datadir + "nugrid/p16_stars.tsv", sep="\s+", comment="#")
P16_stars

In [ ]:
P16_stars.set_index(["mass", "Z"], inplace=True)
P16_stars

In [ ]:
P16_ops_raw = pd.read_csv(datadir + "nugrid/p16_yields.tsv", sep="\t", comment="#", skiprows=[42, 43])
P16_ops_raw

For P16, note that set 1.1 has Z=0.01, set 1.2 has Z=0.02.

In [ ]:
P16_ops_raw["Z"] = np.where(P16_ops_raw.Set == 1.2, 0.02, 0.01)

In [ ]:
P16_ops = P16_ops_raw.melt(id_vars=["Sp", "Z"], value_vars=["OP1.65", "OP2", "OP3", "OP4", "OP5",], var_name="model", value_name="OP")
P16_ops

In [ ]:
P16_ops["M"] = P16_ops.model.str.extract(r"(\d?\.?\d+)")
P16_ops["M"] = pd.to_numeric(P16_ops.M)
P16_ops

In [ ]:
P16 = P16_ops

In [ ]:
m_rem = [P16_stars.loc[r.M, r.Z][0] for i, r in P16.iterrows()]

In [ ]:
P16["Mrem"] = m_rem

In [ ]:
P16["element"] = P16.Sp.str.strip()

In [ ]:
solar_Z.index

In [ ]:
P16[np.isin(P16.element, ["Tc", "Pm"])]

In [ ]:
P16.drop(index=np.where(np.isin(P16.element, ["Tc", "Pm"]))[0], inplace=True)

In [ ]:
Z0 = [solar_Z.loc[r.element].Z  for i, r in P16.iterrows()]

In [ ]:
P16["Zini"] = Z0 * P16.Z / 0.0179

In [ ]:
P16

In [ ]:
P16["mass_yield"] = P16.Zini * (P16.OP - 1) * (1 - P16.Mrem/P16.M)

In [ ]:
plt.scatter(P16.M, P16.Mrem)
plt.xlabel("M ini / msun")
plt.ylabel("reminant mass / msun")

## R 18

In [ ]:
elements = "H, He, Li, B, C, N, O, F, Ne, Na, Mg, Al, Si, P, S, Cl, Ar, K, Ca, Sc, Ti, V, Cr, Mn, Fe, Co, Ni, Cu, Zn, Ga, Ge, As, Se, Br, Kr, Rb, Sr, Y, Zr, Nb, Mo, Ru, Rh, Pd, Ag, Cd, In, Sn, Sb, Te, I, Xe, Cs, Ba, La, Ce, Pr, Nd, Sm, Eu, Gd, Tb, Dy, Ho, Er, Tm, Yb, Lu, Hf, Ta, W, Re, Os, Ir, Pt, Au, Hg, Tl, Pb, Bi".split(", ")

In [ ]:
def read_ritter(filename="element_yield_table_MESAonly_fryer12_delay_total.txt"):
    R18_raw = pd.DataFrame()

    with open(path + "/" + filename) as file:
        skip = 6
        M = None
        M_ej = None
        for line in file:
            if skip > 0:
                skip -= 1
                continue
                
            if line[:7] == "H Table":
                s1, s2 = line[10:].split(",")
                M = float(s1[2:])
                Z = float(s2[2:-2])
                skip = 1
                continue
                
            if line[:8] == "H Mfinal":
                M_rem = float(line[10:])
                M_ej = M - M_rem
                skip = 1
                continue
                
            if M_ej is None:
                continue
            
            _, ele, y, x0, _ = line.split("&")
    
            y = float(y)
            x0 = float(x0)
    
            row = pd.DataFrame({"M":M, "Z":Z, "element":ele.strip(), "Mrem":M_rem, "mass_yield":y, "Zini": x0}, index=[0])
            R18_raw = pd.concat([R18_raw, row], ignore_index=True)
        
    return R18_raw

In [ ]:
def read_ritter_iso(filename="element_yield_table_MESAonly_fryer12_delay_total.txt"):
    R18_raw = pd.DataFrame()

    with open(filename) as file:
        skip = 6
        M = None
        M_ej = None
        for line in file:
            if skip > 0:
                skip -= 1
                continue
                
            if line[:7] == "H Table":
                s1, s2 = line[10:].split(",")
                M = float(s1[2:])
                Z = float(s2[2:-2])
                skip = 1
                continue
                
            if line[:8] == "H Mfinal":
                M_rem = float(line[10:])
                M_ej = M - M_rem
                skip = 1
                continue
                
            if M_ej is None:
                continue
            
            _, ele, y, x0, eleZ, eleA = line.split("&")
    
            y = float(y)
            x0 = float(x0)
    
            row = pd.DataFrame({"M":M, "Z":Z, "isotope":ele.strip(), "Mrem":M_rem, "mass_yield":y, "Zini": x0}, index=[0])
            R18_raw = pd.concat([R18_raw, row], ignore_index=True)
        
    return R18_raw

In [ ]:
R18 = read_ritter_iso("isotope_yield_table_MESAonly_fryer12_delay_winds.txt")
R18["OP"] = R18.mass_yield / ((R18.M -R18.Mrem) * R18.Zini)


In [ ]:
R18.M

In [ ]:
R18_indexed = R18.set_index(["M", "Z", "isotope"])

In [ ]:
R18_indexed.drop_duplicates(inplace=True)

In [ ]:
R18_indexed.sort_index(inplace=True)

In [ ]:
R18_indexed.index

In [ ]:
R18.columns

In [ ]:
R18_mass = R18.groupby(["M", "Z"]).agg({
    "mass_yield": "sum",
    "Mrem": "first"
})

In [ ]:
R18_mass["Mtot"] = R18_mass.mass_yield + R18_mass.Mrem
R18_mass

## B19/21

In [ ]:
b19_raw = pd.read_csv("Battino2019.txt", sep="\s+", skiprows=1,)
b21_raw = pd.read_csv("Battino2021.txt", sep="\s+", skiprows=2)

In [ ]:
np.sum(b19_raw.index.duplicated())

In [ ]:
np.sum(b19_raw[b19_raw.Isotope.str.startswith("K-")], axis=0).iloc[0]

In [ ]:
b19_raw.loc[b19_raw.Isotope == "n-1", "Isotope"] = "Neutron-1"


In [ ]:
B19 = b19_raw.melt(id_vars=["Isotope"],  var_name="model", value_name="mass_yield")

B19["Z"] = 0.
B19["M"] = 0.
B19

B19.loc[B19.model == "m2z1m2", "M"] = 2.
B19.loc[B19.model == "m2z1m2", "Z"] = 0.01
B19.loc[B19.model == "m2z1m2", "Mrem"] = 0.632

B19.loc[B19.model == "m3z1m2", "M"] = 3.
B19.loc[B19.model == "m3z1m2", "Z"] = 0.01
B19.loc[B19.model == "m3z1m2", "Mrem"] = 0.661


B19.loc[B19.model == "m2z2m2", "M"] = 2.
B19.loc[B19.model == "m2z2m2", "Z"] = 0.02
B19.loc[B19.model == "m2z2m2", "Mrem"] = 0.646


B19.loc[B19.model == "m3z2m2", "M"] = 3.
B19.loc[B19.model == "m3z2m2", "Z"] = 0.02
B19.loc[B19.model == "m3z2m2", "Mrem"] = 0.656	
B19["element"] = get_ele(B19.Isotope)

B19.loc[B19.Isotope == "n-1", "Isotope"] = "Neutron-1"
B19[B19.Isotope == "C-12"]

In [ ]:


B19 = B19[B19.M > 0]


# B19["Zini"] = [solar_Z.Z[r.element] * r.Z / Zsun for (i, r) in B19.iterrows()]

# B19["OP"] = B19.mass_yield / ((B19.M - B19.Mrem) * B19.Zini)
B19_indexed = B19.set_index(["M", "Z", "Isotope"])


In [ ]:
B19.Mrem.unique()

In [ ]:
B21 = b21_raw.melt(id_vars=["Isotope"],  var_name="model", value_name="mass_yield")

B21["M"] = 0.
B21["Z"] = 0.

B21.loc[B21.model == "m3z1m3-bigpoc", "M"] = 3.
B21.loc[B21.model == "m3z1m3-bigpoc", "Z"] = 0.001

B21.loc[B21.model == "m2z2m3-bigpoc", "M"] = 2.
B21.loc[B21.model == "m2z2m3-bigpoc", "Z"] = 0.002

B21.loc[B21.model == "m2z1m3-bigpoc", "M"] = 2.
B21.loc[B21.model == "m2z1m3-bigpoc", "Z"] = 0.001


B21 = B21[B21.M > 0]
B21["element"] = get_ele(B21.Isotope)

B21

In [ ]:
B21_indexed = B21.set_index(["M", "Z", "Isotope"])

# Comparisons

In [ ]:
def load_new_table(filename):
    df = pd.read_csv(filename, sep=r"\s+", comment="#")

    df.rename(columns = {"wind_yield": "mass_yield"}, inplace=True)
    df["element"] = get_ele(df.isotope)
    
    return df

In [ ]:

import sys 
sys.path.append("..")

In [ ]:
from arya import arya

In [ ]:
def reproduce_ritter(mass, metallicity, modelname, decay=False):
    if decay:
        filename = f"{modelname}_decay.txt"
    else:
        filename = f"{modelname}.txt"
    r18_new = load_new_table(filename)
    r18 = R18_indexed.loc[mass, metallicity, :]
    plt.scatter(get_A(r18.index.get_level_values(2)), np.log10(r18.mass_yield), label="R18")
    plt.scatter(get_A(r18_new.isotope), np.log10(r18_new.mass_yield),  ec=arya.COLORS[1], lw=0.5, fc="none", label="recalculated")


    plt.ylim(-15, 0)

    plt.legend(bbox_to_anchor=(1,1), loc="upper left")
    plt.title(f"M={mass}, z={metallicity}")

    plt.xlabel("A")
    plt.ylabel("EM")


In [ ]:
def reproduce_ritter(mass, metallicity, modelname, decay=True, ylim=(-1, 2), Xmin=1e-16):
    if decay:
        filename = f"{modelname}_decay.txt"
    else:
        filename = f"{modelname}.txt"
    r18_new = load_new_table(filename)
    r18 = R18_indexed.loc[mass, metallicity, :]
    filt = r18.mass_yield / np.sum(r18.mass_yield) > Xmin
    
    plt.scatter(get_A(r18.index.get_level_values(2))[filt], np.log10(r18.OP[filt]), label="R18")
    
    filt = r18_new.mass_yield / np.sum(r18_new.mass_yield) > Xmin    
    OP = r18_new.mass_yield / np.sum(r18_new.mass_yield) / r18_new.initial_abundance
    
    
    plt.scatter(get_A(r18_new.isotope[filt]), np.log10(OP[filt]),  ec=arya.COLORS[1], lw=0.5, fc="none", label="recalculated")


    plt.ylim(*ylim)

    plt.legend(bbox_to_anchor=(1,1), loc="upper left")
    plt.title(f"M={mass}, z={metallicity}")

    plt.xlabel("A")
    plt.ylabel("log OP")


In [ ]:
reproduce_ritter(2, 0.02, "M2.00Z2.0e-02", ylim=(-0.1, 1), decay=False)

In [ ]:
reproduce_ritter(2, 0.02, "M2.00Z2.0e-02", ylim=(-0.1, 1), decay=True)

In [ ]:
reproduce_ritter(7, 0.001, "M7.000Z0.0010", ylim=(-0.2, 0.5))

In [ ]:
def plot_all_battino(modelname, battinomodelname=None):
    if battinomodelname is None:
        battinomodelname = modelname
    b19_new = load_new_table(f"yields_{modelname}.txt")
    b19_new_decay = load_new_table(f"yields_{modelname}_decay.txt")
    
    try:
        b19 = pd.DataFrame({
            "mass_yield": b19_raw[battinomodelname],
            "isotope": b19_raw.Isotope,
            "element": get_ele(b19_raw.Isotope),
        })
        
        b19.set_index("isotope", inplace=True)
        #B19_indexed.loc[m, z, :]
    except KeyError:
        #b19 = b21_raw[modelname] #B21_indexed.loc[m, z, :]
        b19 = pd.DataFrame({
            "mass_yield": b21_raw[battinomodelname],
            "isotope": b21_raw.Isotope,
            "element": get_ele(b21_raw.Isotope),
        })
        
        b19.set_index("isotope", inplace=True)
    compare_battino_ratios(b19, b19_new)
    plt.title(modelname)
    plt.show()
    
    compare_battino_yields(b19, b19_new)
    plt.title(modelname)
    plt.show()

    reproduce_battino_ratios(b19, b19_new)
    plt.title(modelname)
    plt.show()

    compare_yield_models([b19_new, b19_new_decay], labels=["ejected", "decayed"], 
        kwargss = [dict(), dict(fc="none", ec=arya.COLORS[1], lw=0.8)]
                        )
    plt.title(modelname)
    plt.show()
    
    compare_OP(b19, b19_new)
    plt.title(modelname)
    plt.show()

In [ ]:
def plot_compare_two(modelname, modelname2, **kwargs):
    b19_new = load_new_table(f"yields_{modelname}.txt")
    b19_new_decay = load_new_table(f"yields_{modelname2}.txt")

    compare_yield_models([b19_new, b19_new_decay], labels=[modelname, modelname2],
                          kwargss = [dict(), dict(fc="none", ec=arya.COLORS[1], lw=0.8)],
                         **kwargs
                        )
    plt.show()

In [ ]:
def compare_yield_models(models, labels, kwargss, ylim=(-12, 0), relative=False):

    for i in range(len(models)):
        model  = models[i]
        label = labels[i]
        kwargs = kwargss[i]
        if relative:
            Xf = model.mass_yield / np.sum(model.mass_yield)
            y = np.log10(Xf / model.initial_abundance)
        else:
            y = np.log10(model.mass_yield)
            
        plt.scatter(get_A(model.isotope), y, label=label, **kwargs)

    plt.ylim(*ylim)
    plt.xlabel("mass number")
    if relative:
        plt.ylabel("log OP")
    else:
        plt.ylabel("log EM")

    plt.legend(bbox_to_anchor=(1,1), loc="upper left")

In [ ]:
def compare_OP(b19, model, ylim=(-1, 1.5)):


    Xf = b19.mass_yield / np.sum(b19.mass_yield)
    y = np.log10(Xf / model.initial_abundance.values)
    plt.scatter(get_A(model.isotope.values), y, label="Battino et al.")
    
    Xf = model.mass_yield / np.sum(model.mass_yield)
    y = np.log10(Xf / model.initial_abundance)
    plt.scatter(get_A(model.isotope), y, label="recalculated",  fc="none", ec=arya.COLORS[1], lw=0.8)

    plt.ylim(*ylim)
    plt.xlabel("mass number")
    plt.ylabel("log OP")

    plt.legend(bbox_to_anchor=(1,1), loc="upper left")

In [ ]:
def compare_battino_yields(b19, b19_new):

    
    plt.scatter(get_A(b19.index), np.log10(b19.mass_yield), label="Battino et al.")
    plt.scatter(get_A(b19_new.isotope), np.log10(b19_new.mass_yield), label="recalculated", fc="none", ec=arya.COLORS[1], lw=0.8)

    plt.ylim(-12, 0)
    plt.xlabel("mass number")
    plt.ylabel("log EM")

    plt.legend(bbox_to_anchor=(1,1), loc="upper left")

In [ ]:
def compare_battino_ratios(b19, b19_new):
    
    for (ele, df_new) in b19_new.groupby("element"):
        df = b19[b19.element == ele]
        df_new = df_new.set_index("isotope").loc[df.index]
        
        if len(df) > 0:
            plt.scatter(get_A(df.index), np.log10(df.mass_yield / df_new.mass_yield))
        
    plt.xlabel("mass number")
    plt.ylabel("log EM / EM new")
    plt.ylim(-1, 2)
    #plt.axhline(1, color="k", ls="--")

In [ ]:
def reproduce_battino_ratios(b19, b19_new):
    assert np.all(b19.index == b19_new.isotope)
    plt.scatter(get_A(b19.index), np.log10(b19.mass_yield.values / b19_new.initial_abundance), label="Battino")
    plt.scatter(get_A(b19_new.isotope), np.log10(b19_new.wind_yield_mesaevo / b19_new.initial_abundance), label="reproduced", fc="none", ec=arya.COLORS[1], lw=0.8)

    plt.ylim(-1, 2)
    plt.xlabel("A")
    plt.ylabel("log EM / Xini")

    plt.legend(bbox_to_anchor=(1,1), loc="upper left")

In [ ]:
plot_all_battino("m2z1m2")

In [ ]:
plot_all_battino("m2z1m2")

In [ ]:
plot_all_battino("m3z1m2")

In [ ]:
plot_all_battino("m2z2m2")

In [ ]:
plot_all_battino("m3z2m2")

In [ ]:
plot_all_battino("m3z2m2_hCBM", battinomodelname="m3z2m2-hCBM")

In [ ]:
plot_all_battino("m3z3m2_hCBM", battinomodelname="m3z3m2-hCBM")

In [ ]:
plot_all_battino("m2z1m3", battinomodelname="m2z1m3-bigpoc")

In [ ]:
plot_all_battino("m3z1m3", "m3z1m3-bigpoc")

In [ ]:
plot_all_battino("m2z2m3", "m2z2m3-bigpoc")

In [ ]:
plot_compare_two("m2z2m2_decay", "m2z2m2_Ne22Ad01_decay", relative=True, ylim=(-1, 1))

In [ ]:
plot_compare_two("m3z2m2_hCBM", "m3z2m2")

In [ ]:
plot_compare_two("m3z2m2_hCBM", "m3z2m2", relative=True, ylim=(-1, 1.5))

In [ ]:
plot_compare_two("m3z2m2_hCBM", "m3z2m2-rotmix.st", relative=True, ylim=(-1, 1))

In [ ]:
plot_compare_two("m3z2m2_hCBM", "m3z2m2-rotmix.std2", relative=True, ylim=(-1, 1))

In [ ]:
plot_compare_two("m3z2m2_hCBM", "m3z2m2-rotmix.stx2", relative=True, ylim=(-1, 1))

In [ ]:
plot_compare_two("m3z2m2_hCBM", "m3z2m2-rotmix.std2", relative=True, ylim=(-1, 1))